# Gene Regulatory Network Prediction of Yeast Network
The task:
Predict which transcription factors affects genes (For now this is non-tf)

In [1]:
# Import stuff
import pandas as pd
import numpy as np

In [3]:
# Get expression data
ko_df = pd.read_csv('../dataset/yeast_networks/expression/KO.txt', sep='\t').set_index('Gene')
nv_df = pd.read_csv('../dataset/yeast_networks/expression/NatVar.txt', sep='\t').set_index('Gene')
stress_df = pd.read_csv('../dataset/yeast_networks/expression/Stress.txt', sep='\t').set_index('Name')

# Get Transcription Factors
ko_tf = pd.read_csv('../dataset/yeast_networks/expression/KO_TF_names.txt', sep='\t', header=None)
ko_tf.columns = ['TF']

nv_tf = pd.read_csv('../dataset/yeast_networks/expression/NatVar_TF_names.txt', sep='\t', header=None)
nv_tf.columns = ['TF']

stress_tf = pd.read_csv('../dataset/yeast_networks/expression/Stress_TF_names.txt', sep='\t', header=None)
stress_tf.columns = ['TF']

# Note that there are different gene list for each df, so we will use ko as test for now.

In [5]:
df = ko_df.merge(nv_df, how='inner', left_index=True, right_index=True)
df = df.merge(stress_df, how='inner', left_index=True, right_index=True)

tf = ko_tf.merge(nv_tf, how='inner', left_on='TF', right_on='TF')
tf = tf.merge(stress_tf, how='inner', left_on='TF', right_on='TF')

In [6]:
# Prepare dataset: set which are the predictors (TF), and which are the targets
from sklearn import model_selection
def prep_dataset(target_gene, tf_list, exp_df):
    '''
    Prepares training set and test set for target gene
    
    Args:
        - target_gene: target gene for the iteration (y)
        - exp_df: expression dataframe (already in pandas df format)
        - tf_list: transcription factors, which will be the predictors (X)
        
    Returns:
        - Training and Testing set to be used in model predictions
        - label for predictors, so we can subset this later
    '''
    # Get y (target) and predictor matrix (X)
    y = exp_df.loc[target_gene, :].values
    X = exp_df.loc[tf_list, :]
    
    if target_gene in tf_list.values:
        X = X.drop(index=target_gene)
    
    X_label = X.index # Predictor labels for X
    X = X.values.transpose()
    
    # Split 80:20 for test and train
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)

    
    return X_train, X_test, y_train, y_test, X_label



In [8]:
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

def grn_lasso(target_gene, tf_list, exp_df):
    '''
    GRN inference method using lasso regression
    
    Args:
        - target_gene: target gene for the iteration (y)
        - exp_df: expression dataframe (already in pandas df format)
        - tf_list: transcription factors, which will be the predictors (X)
        
    Returns:
        - Numpy array of type str, with list of non-zero weight predictors
    '''
    # Prep data
    X_train, X_test, y_train, y_test, X_label = prep_dataset(target_gene, tf_list, exp_df)
    
    # Use Lasso regression
    lasso_reg = LassoCV(alphas = [0.01, 0.1, 1, 10, 100], cv=5)
    lasso_reg.fit(X_train, y_train)
    
    # Get scores (R^2)
    train_score = lasso_reg.score(X_train, y_train) # Note: R^2 not very good, maybe use other methods
    test_score = lasso_reg.score(X_test, y_test)
    
    # Get weights of lasso, non zero weights are regulators
    predictors = X_label
    nonzero_filter = lasso_reg.coef_ != 0
    nonzero_preds = predictors[nonzero_filter]
    
    return nonzero_preds.values

def grn_regforest(target_gene, tf_list, exp_df):
    '''
    GRN inference method using regression forest. This method does not assume linearity of data.
    
    Args:
        - target_gene: target gene for the iteration (y)
        - exp_df: expression dataframe (already in pandas df format)
        - tf_list: transcription factors, which will be the predictors (X)
        
    Returns:
        - Numpy array of type str, with list of non-zero weight predictors
        
    '''
    # Prep data
    X_train, X_test, y_train, y_test, X_label = prep_dataset(target_gene, tf_list, exp_df)
    
    # Use regerssion tree
    forest_reg = RandomForestRegressor(n_estimators = 10, max_depth = 8, bootstrap = True, min_samples_leaf = 10, n_jobs=-1)
    forest_reg.fit(X_train, y_train)
    
    # Get Scores (R^2)
    train_score = forest_reg.score(X_train, y_train)
    test_score = forest_reg.score(X_test, y_test)
    
    # Get feature importance
    predictors = X_label
    nonzero_filter = forest_reg.feature_importances_ != 0
    nonzero_preds = predictors[nonzero_filter]
    
    return nonzero_preds.values



# Read in an expression file, and predict GRN using a method
def predict_grn(exp_df, tf_list, method, *args):
    '''
    Function that reads in expression level and predict its GRN using a specific method
    
    Args:
        - exp_df: expression dataframe (already in pandas df format), index is gene names, columns are sample / treatments
        - method: Method of inferring GRN (Example: grn_lasso)
        - *args: additional arguments for method
        - tf_list: transcription factors, which will be the predictors (X)
        
    Returns:
        - Set of predicted edges with format (regulator(space)target)
    '''
    
    pred_edges = set()

    for target in exp_df.index.values:
        # Get regulators for every target gene

        pred_regs = method(target, tf_list, exp_df, *args)

        # Add the predicted regulators -> target edge to the predicted edges
        for reg in pred_regs:
            pred_edges.add(f'{reg}->{target}')
    
    return pred_edges

In [11]:
# Performance Metrics

def iou_score(gold_df, pred_grn_edges):
    '''
    Function to score predicted grn vs gold standard based on intersection over union.

    Score:
    For starters, we will use the simple score of intersection / union
    Intersection: Count of edges in both gold standard AND predicted grn
    Union: Count of gold standard edges + predicted grn edges - Intersection

    Intersection / Union is the score.

    Args:
        - gold_file: Path to gold standard file
        - pred_grn_edges: set of predicted grn edges

    Returns:
        - IOU score
    '''

    # Set of gold standard edges
    gold_edge_set = set(gold_df.loc[:, 'Regulator'] + '->' + gold_df.loc[:, 'Target'])

    gold_regulators = set(gold_df.loc[:, 'Regulator'])
    pred_grn_edges = [edge for edge in pred_grn_edges if edge.split('->')[0] in gold_regulators]

    # Get Intersection and Union
    intersection = gold_edge_set.intersection(pred_grn_edges)
    union = gold_edge_set.union(pred_grn_edges)

    # Get score: Intersection / Union
    iou_score = len(intersection) / len(union)
    print(f'Union edges count: {len(union)}')
    print(f'Intersection edges count: {len(intersection)}')
        
    return iou_score

In [9]:
# Predict Edges
import time

time_0 = time.time()

regforest_edges = predict_grn(df, tf.loc[:, 'TF'], grn_regforest)
time_1 = time.time()
print("Time Training RegForest: ", time.strftime("%H:%M:%S", time.gmtime(time_1 - time_0)))

lasso_edges = predict_grn(df, tf.loc[:, 'TF'], grn_lasso)
time_2 = time.time()
print("Time Training Lasso: ", time.strftime("%H:%M:%S", time.gmtime(time_2 - time_0)))

Time Training RegForest:  01:15:00


/Users/SamYang95/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6516312924229624, tolerance: 0.7359066690891514
  tol, rng, random, positive)
/Users/SamYang95/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.0589834671579865, tolerance: 1.0344055809929955
  tol, rng, random, positive)
/Users/SamYang95/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.312769507908342, tolerance: 1.0388831425471838
  tol, rng, random, positive)
/Users/SamYang95/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: Con

/Users/SamYang95/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.522625991355198, tolerance: 1.2708616753339854
  tol, rng, random, positive)
/Users/SamYang95/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18.02415833342866, tolerance: 1.4407523029457638
  tol, rng, random, positive)
/Users/SamYang95/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.1238733866971415, tolerance: 1.3524376594242564
  tol, rng, random, positive)
/Users/SamYang95/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: Conv

Time Training Lasso:  01:35:49


In [12]:
ko_gold_df = pd.read_csv("../dataset/yeast_networks/gold/MacIsaac2.KO.txt", sep = '\t', header = None, names = ['Regulator', 'Target'])
nv_gold_df = pd.read_csv("../dataset/yeast_networks/gold/MacIsaac2.NatVar.txt", sep = '\t', header = None, names = ['Regulator', 'Target'])
stress_gold_df = pd.read_csv("../dataset/yeast_networks/gold/MacIsaac2.Stress.txt", sep = '\t', header = None, names = ['Regulator', 'Target'])

gold_df = ko_gold_df.merge(nv_gold_df, how='outer', left_on=['Regulator', 'Target'], right_on=['Regulator', 'Target'])
gold_df = gold_df.merge(stress_gold_df, how='outer', left_on=['Regulator', 'Target'], right_on=['Regulator', 'Target'])

In [20]:
FinallyItIsDone()

In [16]:
lasso_score = iou_score(gold_df, lasso_edges)
regforest_score = iou_score(gold_df, regforest_edges)

Union edges count: 206278
Intersection edges count: 1394
Union edges count: 238909
Intersection edges count: 1509


In [18]:
print(f'KO Lasso score: {lasso_score}') # Much better than last time in try 1
print(f'KO Regforest score: {regforest_score}') 

KO Lasso score: 0.006757870446678754
KO Regforest score: 0.006316212449091495


In [19]:
# Sound alert after code is finished!
from IPython.display import Audio, display
def FinallyItIsDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))